In [8]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import os
import logging

In [9]:
#os.chdir('..')
os.chdir('C:/Users/BC966HL/OneDrive - EY/4. Mestrado/Dev/')
os.getcwd()

'C:\\Users\\BC966HL\\OneDrive - EY\\4. Mestrado\\Dev'

In [10]:
from Coding.etl.data_normalization import data_norm
from Coding.etl.input_parameters import (recipe_def, create_cost_column, input_lots, target_volume, model_files)
from Coding.model.minimize import (model_input_read, constraints_bounds, cost_obj_f, minimize_round, run_model, model_results)

In [11]:
# Reading CQI CSV file as input, normalizing data and creating cleaned_cqi_file.csv with no-NaN rows and consistent coffee sample parameters
data_norm()

# recipe_def function to read the cleaned_cqi_file.csv and extract a random row to be used as target for the model. It will return a single row dataframe with unusable LOT_AVAILABILITY_KG field and the normalized dataframe with the selected row removed
df_recipe, df_norm = recipe_def()

# create_cost_column gets the normalized dataframe without the recipe row and create a random cost per kg in BRL to the lot on the row, the model will seek to minimize this final cost attending to the other parameters
df_norm_cost = create_cost_column(df_norm)

# input_lots will get the normalized dataframe without the recipe row and with the cost column and extracts from the n available rows a lot_limit (default = 10) sized dataframe which will be used as available lots to the model work on
df_lots_available = input_lots(df_norm_cost, return_df_norm_wout_lots = False, lot_limit = 10)

# df_recipe_volume simulate the batch volume
# if simulate_broken_volume is set to True, it will create a volume higher than the available lots can provide together, reaching from the volume sum to broken_perc*volume sum, the broken_perc is set to 1.5 as default
# if is set to False, will get from the lots available volume sum * low_perc to volume sum * high perc, which are set as default to 0.2 and 0.8 respectively
df_recipe_volume = target_volume(df_recipe, df_lots_available, simulate_broken_volume = False)

# model_files export the recipe and available lots datasets to /Datasets/model as input_recipe.csv and input_lots_available.csv with ';' as CSV separator
model_files(df_recipe_volume, df_lots_available)

In [12]:
df_recipe, df_available_lots = model_input_read()

#constraints, bnd = constraints_bounds(df_recipe, df_available_lots, ub = 0.7)

In [13]:
{str(model_iteration): model_results(df_recipe, df_available_lots, model_output)}

In [14]:
model_iteration = 0
simulation_list = []
constraints, bnd = constraints_bounds(df_recipe, df_available_lots, ub = 0.7)
for iteration in range(20):
    try:
        model_iteration += 10
        model_output = run_model(df_recipe, df_available_lots, constraints, bnd, times = model_iteration)
        simulation_list.append(model_results(df_recipe, df_available_lots, model_output))
        logging.warning('Appended with iteration = %d', model_iteration)
    except:
        logging.warning('Model could not find a solution within range and iterations')

        # Avaliar # iterations vs resultado da função objetivo
        # https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d_radial.html
        # https://matplotlib.org/gallery/mplot3d/voxels_torus.html#sphx-glr-gallery-mplot3d-voxels-torus-py

In [15]:
simulation_list

[(   LOT_AVAILABILITY_KG  MODEL_RESULT_ARRAY  ALLOCATION_TEST
  0                 20.0                   0             True
  1                 18.0                  17             True
  2                300.0                   0             True
  3                 35.0                  35             True
  4               5520.0                5520             True
  5               3000.0                   0             True
  6              17500.0                4177             True
  7              18975.0               16658             True
  8                600.0                 599             True
  9               1725.0                   0             True,
        AROMA    FLAVOR  AFTERTASTE   ACIDITY      BODY   BALANCE  MOISTURE
  0  0.101718  0.073297    0.030293  0.241663  0.224236  0.130311      0.01,
        AROMA    FLAVOR  AFTERTASTE   ACIDITY      BODY   BALANCE  MOISTURE
  0  1.356236  0.966977    0.408259  3.188167  2.958253  1.737476  8.333333),
 (   LOT_A